In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
np.set_printoptions(precision=2)
from cycler import cycler
plt.rcParams.update(plt.rcParamsDefault)
from pathlib import Path

import wave as w
import constants as c
import material as m

# Example usage of simulated materials
This notebook is ment as an reference on how to use `PlaneWave` and `Material` classes,
and for visual test of the implementation.

In [ ]:
figures_dir = "figures/"
Path(figures_dir).mkdir(parents=True, exist_ok=True)

### Generate an example spectrum
Just an example how to create and plot wave.

In [ ]:
s = w.GaussianPlanarWave(mean=c.GREEN, std=(5*c.MICRO))
s_replay = w.WhitePlanarWave()
z = np.linspace(-6*c.MICRO, 6*c.MICRO, 1000)
s.delay(0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
s.plot(ax=ax1, spectrum_axis=ax1.twinx(), wavelength=True)
s.plot_amplitude(z, ax=ax2)
ax2.legend()
fig.tight_layout()
plt.savefig(figures_dir + "spectrum.pdf", transparent=True)
plt.show()

### Compare different materials

#### Analitical vs constant vs matrix theory
Create the same material (empty space) using three different material models in oder to inspect that they give the same resut.

In [ ]:
air = m.EmptySpace(z=z, name="analytic")
common_n = 1
plate = m.PhotoSensitiveMaterial(z=z, r=0, name="transfer matrices")
dielectric = m.SimpleDielectric(z=z, n0=1, name="transfer matrices")

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

air.plot(ax1, imaginary_axis=ax1.twinx())
plate.plot(ax2, imaginary_axis=ax2.twinx())
dielectric.plot(ax3, imaginary_axis=ax3.twinx())
fig.tight_layout() 
plt.show()

Calculate interference pattern in those three materials: again we can see that the patterns match. 
The index of refraction after recording would be different in those materials because ConstantMaterial's index of refraction cannot be modified.

In [ ]:
fig, (ax1) = plt.subplots(1, 1, figsize=(8,4))

dielectric.record(s,s)
dielectric.plot_recent_energy(ax1, label="refraction")
plate.record(s,s)
plate.plot_recent_energy(ax1, label="reflection", alpha=0.5)
ax1.legend()
plt.show()

### An example recording

Left: interference pattern 
created in the material. Middle: index of 
refraction after recording. Right: reflected pulse, when the 
incoming pulse was the same as the recorded.

In [ ]:
z = np.linspace(0, 5*c.MICRO, 1000)
dielectric = m.PhotoSensitiveMaterial(z=z, max_dr=0.005, r=0, name="transfer matrices")
s0 = w.GaussianPlanarWave(mean=c.GREEN, std=(3*c.MICRO))
s1 = w.GaussianPlanarWave(mean=c.GREEN, std=(3*c.MICRO))
s1.shift(z=-5*c.MICRO)
dielectric.record(s0, s1)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
dielectric.plot_recent_energy(ax1)
ax1.set_title("interference")
dielectric.plot(ax=ax2)
dielectric.reflect(s_replay).plot(ax3, wavelength=False)
plt.tight_layout()
plt.savefig(figures_dir + "recording.pdf", transparent=True)
plt.show()